<a href="https://colab.research.google.com/github/SeohyunLyoo/Study/blob/main/Attention%20-%20Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import nltk
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My Drive/Study


Mounted at /content/drive/
/content/drive/My Drive/Study


In [10]:
!ls

ChatBotData.csv  Reviews.csv


# **1. 데이터 준비 및 불용어 취합**

In [11]:
data = pd.read_csv('Reviews.csv', nrows = 100000)
print(len(data)), print(data.info())

100000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      100000 non-null  int64 
 1   ProductId               100000 non-null  object
 2   UserId                  100000 non-null  object
 3   ProfileName             99994 non-null   object
 4   HelpfulnessNumerator    100000 non-null  int64 
 5   HelpfulnessDenominator  100000 non-null  int64 
 6   Score                   100000 non-null  int64 
 7   Time                    100000 non-null  int64 
 8   Summary                 99998 non-null   object
 9   Text                    100000 non-null  object
dtypes: int64(5), object(5)
memory usage: 7.6+ MB
None


(None, None)

In [12]:
data.isnull().sum()

,0
Id,0
ProductId,0
UserId,0
ProfileName,6
HelpfulnessNumerator,0
HelpfulnessDenominator,0
Score,0
Time,0
Summary,2
Text,0


In [13]:
df = data[['Text', 'Summary']]
df.head()

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy


In [17]:
### Text 및 Summary 항목 중복 여부 체크
df['Text'].nunique(), df['Summary'].nunique()

(88426, 72348)

In [21]:
### Text열 중복 데이터 삭제 (Summary는 중복 데이터 유지, Text에 대한 동일 Summary 지원)
df.drop_duplicates(subset = ['Text'], inplace = True)
print(len(df))

88426


In [24]:
### Text 및 Summary Null 값 여부 확인
df['Text'].isnull().sum(), df['Summary'].isnull().sum()

(0, 1)

In [25]:
### Summary 항목 null 값인 데이터 삭제
df.dropna(axis=0, inplace=True)
df['Text'].isnull().sum(), df['Summary'].isnull().sum()

(0, 0)

In [29]:
### NLTK 불용어 취합
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [32]:
print(stop_words)

{"wasn't", 'more', 'on', 'than', 'won', 'over', 'so', 'below', 'needn', 'my', "wouldn't", 'to', 'ours', 'from', 'o', 'ourselves', 'up', 'until', 'any', 'at', 'being', 'all', 'both', 'weren', 'will', 'an', 'just', "haven't", 'not', 'this', 'do', 'it', 'ma', 'have', "aren't", "couldn't", 'doesn', 'or', 'their', 'by', "you're", "it's", 'once', 'now', 'again', "won't", 'into', 'off', 'where', 'couldn', 'before', 'yourselves', 'against', 'out', 'few', 'wouldn', 'nor', 'with', 'who', 'shouldn', 'does', 'some', "don't", 'having', 'when', 'there', 'i', 'while', 'isn', 'yourself', 'itself', 'aren', 'here', 'other', 'only', 'hers', "weren't", 'that', 'own', 's', 'm', 'you', 'those', 'haven', 'but', "you've", 'ain', 'how', 'be', 'me', 'why', 'in', 'mustn', 'yours', 'and', 'doing', 'your', 'because', 't', 'if', "doesn't", 'after', 'did', 'further', 'through', 'him', 'herself', 'are', 'can', 'll', 'her', 'as', 'don', 'the', 'no', "shan't", 'under', 'too', 'himself', "you'll", 'she', "didn't", 'them

# **2. 데이터 전처리**